In [3]:
import pandas as pd
import numpy as np

from yaml import safe_load

import os

## tqdm is for seeing the prgresss
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

pd.pandas.set_option("Display.max_columns",None)

##  yml to dataframe

In [ ]:
filenames = []
for file in os.listdir('Data'):
    filenames.append(os.path.join("Data",file))

In [ ]:
df = pd.DataFrame()
counter = 1
for file in tqdm(filenames):
    with open(file,'r') as f:
        df_temp = pd.json_normalize(safe_load(f))
        df_temp['match_id'] = counter
        df = df.append(df_temp)
        counter+=1

In [ ]:
df.to_csv('ymal_to_dataframe.csv')

In [ ]:
backup = df.copy()

In [ ]:
df.head()

In [ ]:
column = ['match_id','innings','info.city','info.competition','info.gender','info.match_type','info.overs','info.teams','info.venue']
df = df[column]

In [ ]:
df.head()

In [ ]:
df = df.drop(['info.competition','info.match_type','info.overs','info.gender'],axis=1)

In [ ]:
df.head()

In [ ]:
## getting bowl by bowl data

count = 1
delivery_df = pd.DataFrame()
for index , row in df.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,784,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        
          for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append(0)
    loop_df = pd.DataFrame({
        'match_id':match_id,
        'teams':teams,
        'batting_team':batting_team,
        'ball':ball_of_match,
        'batsman':batsman,
        'bowler':bowler,
        'runs':runs,
        'player_dismissed':player_of_dismissed,
        'city':city,
        'venue':venue,
    })

    delivery_df = delivery_df.append(loop_df)


In [ ]:
delivery_df.head()

## Data form we want for Model
### batting_team   bowling_team   current_score   wicket_left   current_runrate   city   balls_left   runs_last_five_overs

In [ ]:
## getting bowler team
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team
        
delivery_df['bowling_team'] = delivery_df.apply(bowl,axis=1)
delivery_df = delivery_df.drop(['teams'],axis=1)

In [ ]:
delivery_df.head()

In [ ]:
### keepig only the current ipl team
eligable_teams = ['Mumbai Indians','Kings XI Punjab','Royal Challengers Bangalore','Chennai Super Kings',
                  'Kolkata Knight Riders','Sunrisers Hyderabad','Rajasthan Royals','Delhi Capitals']

In [ ]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(eligable_teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(eligable_teams)]

In [ ]:
delivery_df.to_csv("main_cleaned.csv")

In [ ]:
df = pd.read_csv("main_cleaned.csv")
df = df.drop(['Unnamed: 0'],axis=1)

In [ ]:
# null values
df.isnull().sum()

In [ ]:
## filling city value with help of venue
## df[df['city'].isnull()]['venue'].str.split().apply(lambda x: x[0])
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x: x[0]),df['city'])
df['city'] = cities
df = df.drop(['venue'],axis=1)

In [ ]:
df.head()

In [ ]:
## city has many values we need to fix it
## we simpaly take the city where atleast 5 matches are played
## means where balls > 600 , means 5 matches played

eligable_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index
df = df[df['city'].isin(eligable_cities)]

In [ ]:
df.head()

In [ ]:
## current score
df['current_score'] = df.groupby('match_id').cumsum()['runs']

In [ ]:
## balls left
df['over'] = df['ball'].apply(lambda x : str(x).split('.')[0])
df['ball_no'] = df['ball'].apply(lambda x : str(x).split('.')[1])

In [ ]:
## how many bolws are thrown
df['balls_bowled']  = df['over'].astype('int')*6 + df['ball_no'].astype('int')

In [ ]:
## balls left
df['balls_left'] = 120 - df['balls_bowled']
## because extra
df['balls_left'] = df['balls_left'].apply(lambda x: 0 if x<0 else x)

In [ ]:
df.head()

In [ ]:
## wicket left
df['player_dismissed'] = df['player_dismissed'].apply(lambda x : 0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype(int)
df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']
df['wicket_left'] = 10 -df['player_dismissed']

In [ ]:
df.head()

In [ ]:
## current run rate
df['crr'] = (df['current_score']*6)/df['balls_bowled']

In [ ]:
### last five over runs
## window 30 because 5*6=30 , 5 over 6 balls
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_Five = []
for id in match_ids:
    last_Five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())

In [ ]:
df['last_five'] = last_Five

In [ ]:
df.head()

In [ ]:
## final score in this match
df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [ ]:
df.head()

In [ ]:
df = df[['batting_team','bowling_team','city','current_score','balls_left','wicket_left','crr','last_five','runs_x']]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
## because ouer data not goes bised so we are doing suffling
df = df.sample(df.shape[0])

In [ ]:
df.head()

In [ ]:
df.to_csv("last_csv_data.csv")

## train test split

In [4]:
new_df = pd.read_csv("last_csv_data.csv")

In [5]:
new_df = new_df.drop(['Unnamed: 0'],axis=1)

In [6]:
X = new_df.drop(['runs_x'],axis=1)
y = new_df['runs_x']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [8]:
X_test.head()

,batting_team,bowling_team,city,current_score,balls_left,wicket_left,crr,last_five
18987,Sunrisers Hyderabad,Chennai Super Kings,Pune,171,3,6,8.769231,45.0
21409,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata,46,85,9,7.885714,32.0
22681,Chennai Super Kings,Royal Challengers Bangalore,Bangalore,47,77,9,6.558140,31.0
27659,Royal Challengers Bangalore,Chennai Super Kings,Bangalore,95,41,7,7.215190,36.0
35831,Royal Challengers Bangalore,Kings XI Punjab,Chandigarh,129,10,3,7.036364,37.0


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score , mean_absolute_error

In [10]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
],
remainder='passthrough'
)

In [ ]:
## for XGBoost
pipe_xgb = Pipeline(steps=[
    ('column_transfomer',trf),
    ('standaradiztion',StandardScaler()),
    ('model',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

pipe_xgb.fit(X_train,y_train)

y_pred = pipe_xgb.predict(X_test)
print(f"train score: {pipe_xgb.score(X_train,y_train)}")
print(f"test r2 score: {r2_score(y_test,y_pred)}")
print(f"mae: {mean_absolute_error(y_test,y_pred)}")

In [12]:
## radnom Forest
pipe_rf = Pipeline(steps=[
    ('column_transfomer',trf),
    ('standaradiztion',StandardScaler()),
    ('model',RandomForestRegressor(n_estimators=1000,max_depth=12,random_state=1))
])

pipe_rf.fit(X_train,y_train)

y_pred = pipe_rf.predict(X_test)
print(f"train score: {pipe_rf.score(X_train,y_train)}")
print(f"test r2 score: {r2_score(y_test,y_pred)}")
print(f"mae: {mean_absolute_error(y_test,y_pred)}")

train score: 0.8784125126990758
test r2 score: 0.856333300598374
mae: 7.870687984533939


In [ ]:
### linear regressor
pipe_lr = Pipeline(steps=[
    ('column_transfomer',trf),
    ('standaradiztion',StandardScaler()),
    ('model',LinearRegression())
])

pipe_lr.fit(X_train,y_train)

y_pred = pipe_lr.predict(X_test)
print(f"train score: {pipe_lr.score(X_train,y_train)}")
print(f"test r2 score: {r2_score(y_test,y_pred)}")
print(f"mae: {mean_absolute_error(y_test,y_pred)}")

In [ ]:
### SVM
pipe_svm = Pipeline(steps=[
    ('column_transfomer',trf),
    ('standaradiztion',StandardScaler()),
    ('model',SVR())
])

pipe_svm.fit(X_train,y_train)

y_pred = pipe_svm.predict(X_test)
print(f"train score: {pipe_svm.score(X_train,y_train)}")
print(f"test r2 score: {r2_score(y_test,y_pred)}")
print(f"mae: {mean_absolute_error(y_test,y_pred)}")

In [ ]:
import pickle
pickle.dump(pipe_xgb,open('Ipl_xgboostRegressor_model.pkl','wb'))
pickle.dump(pipe_rf,open('Ipl_randomforest_model.pkl','wb'))

In [13]:
import joblib
joblib.dump(pipe_rf,'Ipl_randomforest_model.pkl')

['Ipl_randomforest_model.pkl']

In [14]:
model = joblib.load('Ipl_randomforest_model.pkl') 

In [23]:
df_to_pred = pd.DataFrame([['Rajasthan Royals','Kings XI Punjab','Chandigarh',148,25,7,7.03,49]],
             columns=['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left',
       'wicket_left', 'crr', 'last_five'])

In [24]:
predicated = model.predict(df_to_pred)
predicated[0]

150.55888462186414